In [5]:
import sys
sys.path.append('../ppda')

import pandas as pd
from datetime import datetime
import re

from db_service import DBService
import config

print("Connecting to ",config.BASE_URL)
conn = DBService()
conn.login('administrator', 'wellspring@2024')
print('Done!')

Connecting to  https://dev.wellspringsaigon.edu.vn:80


SSLError: HTTPSConnectionPool(host='dev.wellspringsaigon.edu.vn', port=8000): Max retries exceeded with url: / (Caused by SSLError(SSLError(1, '[SSL] record layer failure (_ssl.c:997)')))

In [2]:
import unicodedata

from sis_school_year import SISSchoolYear

from sis_person import SISPerson
from sis_student import SISStudent
from sis_school_class import SISSchoolClass
from sis_school_grade_level import SISSchoolGradeLevel
from sis_family import SISFamily
from sis_family_guardian import SISFamilyGuardian
from sis_family_child import SISFamilyChild

from sis_staff import SISStaff
from sis_timetable import SISTimetable
from sis_timetable_column import SISTimetableColumn
from sis_timetable_day import SISTimetableDay
from sis_timetable_column_row import SISTimetableColumnRow

from sis_course import SISCourse
from sis_course_class import SISCourseClass
from sis_attendance_log_course_class import SISAttendanceLogCourseClass
from sis_attendance_log_school_class import SISAttendanceLogSchoolClass
from sis_class_feed import SISClassFeed
from pp_user import PPUser

def delete_all(cls):
    print('Deleting all', cls.doctype)
    while True:
        data = cls.find(limit_page_length=500)
        if len(data) == 0:
            break
        for id in data['name'].values:
            if cls.doctype == 'SIS Person':
                try:
                    cls.delete_by_id(id)
                except:
                    continue
            else:
                cls.delete_by_id(id)
        print('Deleted', len(data))
    print('-'*20)

def reset_db():
    delete_all(SISAttendanceLogSchoolClass)
    delete_all(SISAttendanceLogCourseClass)
    delete_all(SISClassFeed)
    delete_all(SISCourseClass)
    delete_all(SISCourse)
    delete_all(SISSchoolClass)
    delete_all(SISFamily)
    delete_all(SISStudent)
    delete_all(SISStaff)
    delete_all(SISPerson)

def split_name(full_name):
    try:
        last_name = full_name.split(' ', 1)[0].strip()
        first_name = full_name.split(' ', 1)[1].strip()
        return first_name, last_name
    except Exception as e:
        print(f"ERROR split_name: {full_name}")
        return None, None
    

# Function to normalize data
def normalize_unicode(data, form='NFC'):
    return unicodedata.normalize(form, data) if pd.notna(data) else None

def normalize_df(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.strip()
            df[col] = df[col].apply(normalize_unicode)
    return df

def extract_school_class_from_text(text):
    """
    Using regex to extract school class from text.. School class is in the format: [Grade].[Class order] e.g. 1.1, 6.10

    Text is a string that could has multiple lines. The function should return the first school class found in the text.
    """
    text = text.replace('\n', ' ')
    match = re.search(r'\b\d+\.\d+\b', text)
    if match:
        return match.group()
    return None

def get_timetable_day(timetable_days, text):
    vn_translations = {
        "Hai": "Mon",
        "Ba": "Tue",
        "Tư": "Wed",
        "Năm": "Thu",
        "Sáu": "Fri",
    }
    en_translations = {
        "Monday": "Mon",
        "Tuesday": "Tue",
        "Wednesday": "Wed",
        "Thursday": "Thu",
        "Friday": "Fri",
    }
    if text in vn_translations.keys():
        translated_weekday = vn_translations[text]
    elif text in en_translations.keys():
        translated_weekday = en_translations[text]
    else:
        raise ValueError(f"Invalid weekday: {text}")

    for timetable_day in timetable_days:
        if timetable_day["weekday"] == translated_weekday:
            return timetable_day
        
    return None
# reset_db()

In [3]:
# Get current school year

CUR_SCHOOL_YEAR_ID = SISSchoolYear.find_one(filters={"status": "Current"}).name

### Academic Year Event

In [ ]:
from sis_academic_year_event import SISAcademicYearEvent

# Get all academic year events
aca_event_df = pd.read_excel('../input_data/SIS Academic Year Events.xlsx', engine="openpyxl")
aca_event_df.head()

for index, row in aca_event_df.iterrows():
    event = SISAcademicYearEvent({
        "start_date": datetime.strftime(row['Start Date'], "%Y-%m-%d"),
        "end_date": datetime.strftime(row['End Date'], "%Y-%m-%d"),
        "school_year": CUR_SCHOOL_YEAR_ID,
        "description": row['Description'] if not pd.isna(row['Description']) else "",
        "title": row['Title'],
    })
    event.save_if_not_exists(filters={"title": row['Title'], "school_year": CUR_SCHOOL_YEAR_ID})

### Student, Guardian, School Class, Family

In [101]:
# Load data student, parent, school class
import pandas as pd

df = pd.read_excel('../input_data/FINALIZED_ADMISSION_2324_DATA.xlsx', engine="openpyxl")
df['std_full_name'] = df['std_full_name'].str.strip()
df['mother_full_name'] = df['mother_full_name'].str.strip()
df['mother_email'] = df['mother_email'].str.strip()
df['father_full_name'] = df['father_full_name'].str.strip()
df['father_email'] = df['father_email'].str.strip()
df['std_2324_grade'] = df['std_2324_grade'].astype(str)

In [102]:
df.columns

Index(['wssg_std_code', 'std_full_name', 'std_first_name', 'std_dob',
       'std_gender', 'std_2223_grade', 'std_2324_grade', 'std_old_school',
       'std_note', 'address', 'mother_full_name', 'mother_email',
       'mother_mobile_nr_1', 'mother_mobile_nr_2', 'father_full_name',
       'father_email', 'father_mobile_nr_1', 'father_mobile_nr_2', 'job_title',
       'company', 'application_form', 'bod', 'sub_email_1', 'sub_email_2'],
      dtype='object')

In [ ]:
df[df['std_full_name'].apply(lambda x: len(x.split(' ')) if type(x) == str else 0) == 1]

In [ ]:
df.info()

In [ ]:
df['std_2324_grade'].apply(lambda x: x.split('.')[0].split(' ')[1]).unique()

In [ ]:
# Validate data
# wssg_std_code must be unique
assert df['wssg_std_code'].is_unique, "wssg_std_code is not unique"

# print duplicate std_full_name
# duplicate_std_full_name = df[df.duplicated('std_full_name')]
# if not duplicate_std_full_name.empty:
#     print("Duplicate std_full_name")
#     print(duplicate_std_full_name)

In [ ]:
# Delete all family
families = SISFamily.find(limit_page_length=3000)
for id in families['name'].values:
    SISFamily.delete_by_id(id)

In [ ]:






grade_levels = []
for grade in range(1, 13):
    grade_level = SISSchoolGradeLevel({
        "title": f"Grade {grade}",
        "short_title": f"G{grade}",
        "sequence_number": grade
    })
    grade_level.save_if_not_exists(filters={"title": f"Grade {grade}"})
    grade_levels.append(grade_level)

for index, row in df.iterrows():
    print("Executing row ", index)
    # Create SIS Person for student, if not exists
    first_name, last_name = split_name(row['std_full_name'])
    std_person = SISPerson({
        "first_name": first_name,
        "last_name": last_name,
        "email": None,
        "phone_number": None,
        "gender": row["std_gender"],
        "date_of_birth": datetime.strftime(row["std_dob"], "%Y-%m-%d"),
        "primary_role": "Student",
    })
    std_person_exists = std_person.save_if_not_exists(filters={
        "first_name": first_name, 
        "last_name": last_name, 
        "date_of_birth": datetime.strftime(row["std_dob"], "%Y-%m-%d")
    })

    # Create SIS Student with person id
    sis_student = SISStudent({
        "person": std_person.name,
        "wellspring_student_code": row["wssg_std_code"],
    })
    sis_student.save_if_not_exists(filters={
        "wellspring_student_code": row["wssg_std_code"]
    })

    # Create SIS School Class
    try:
        grade_level_index = row['std_2324_grade'].split('.')[0].split(' ')[1]
        grade_level_id = grade_levels[int(grade_level_index) - 1].name
        sis_school_class = SISSchoolClass({
            "school_year": CUR_SCHOOL_YEAR_ID,
            "school_grade_level": grade_level_id,
            "title": row['std_2324_grade'],
            "short_title": row['std_2324_grade'].split(' ')[1],
        })
        sis_school_class_exists = sis_school_class.save_if_not_exists(filters={
            "title": row['std_2324_grade'],
            "school_year": CUR_SCHOOL_YEAR_ID,
        })
        if sis_school_class_exists:
            sis_school_class = SISSchoolClass.find_by_id(sis_school_class_exists)
    except Exception as e:
        print("ERROR importing School Class:")
        print({
            "school_year": CUR_SCHOOL_YEAR_ID,
            "school_grade_level": grade_level_id,
            "title": row['std_2324_grade'],
            "short_title": row['std_2324_grade'].split(' ')[1],
        })
        break
   

    # Enroll student to school class
    if not hasattr(sis_school_class, "participants"):
        setattr(sis_school_class, "participants", [])
        print("------------Create participants")
    std_already_enrolled = False
    for participant in sis_school_class.participants:
        if participant["person"] == std_person.name:
            std_already_enrolled = True
            break
    if not std_already_enrolled:
        sis_school_class.participants.append({
            "person": std_person.name,
            "role": "Student",
        })
        sis_school_class.save()
        print(f"----------Enroll student to school class {sis_school_class.title} - {std_person.last_name} {std_person.first_name}")

    # Create SIS Person for mother, if not exists
    has_mother = pd.notna(row['mother_full_name'])
    if has_mother:
        mother_first_name, mother_last_name = split_name(row['mother_full_name'])
        mother_person = SISPerson({
            "first_name": mother_first_name,
            "last_name": mother_last_name,
            "email": row["mother_email"] if pd.notna(row["mother_email"]) else None,
            "phone_number": row["mother_mobile_nr_1"] if pd.notna(row["mother_mobile_nr_1"]) else None,
            "gender": "Female",
            "date_of_birth": None,
            "primary_role": "Guardian",
        })

        mother_exists = mother_person.save_if_not_exists(filters={
            "first_name": mother_first_name, 
            "last_name": mother_last_name, 
            "phone_number": row["mother_mobile_nr_1"] if pd.notna(row["mother_mobile_nr_1"]) else None,
        })

    # Create SIS Person for father, if not exists
    has_father = pd.notna(row['father_full_name'])
    if has_father:    
        father_first_name, father_last_name = split_name(row['father_full_name'])
        father_person = SISPerson({
            "first_name": father_first_name,
            "last_name": father_last_name,
            "email": row["father_email"] if pd.notna(row["father_email"]) else None,
            "phone_number": row["father_mobile_nr_1"] if pd.notna(row["father_mobile_nr_1"]) else None,
            "gender": "Male",
            "date_of_birth": None,
            "primary_role": "Guardian",
        })
        father_exists = father_person.save_if_not_exists(filters={
            "first_name": father_first_name, 
            "last_name": father_last_name, 
            "phone_number": row["father_mobile_nr_1"] if pd.notna(row["father_mobile_nr_1"]) else None,
        })

    

    families = SISFamily.find(filters={"home_address": row["address"]}, output="Object")
    if len(families) > 1:
        print("Address Duplicate")

    family_found = False
    for family in families:
        sis_family = SISFamily.find_by_id(family.name)
        if (has_mother and (sis_family.guardian_exists(mother_exists)) or (has_father and (sis_family.guardian_exists(father_exists)))):
            if not sis_family.child_exists(std_person.name):
                sis_family.children.append({"person": std_person.name})
                sis_family.save()
                print("add student to family", sis_family.home_address)
            family_found = True
            break
        
    if not family_found:
        guardians = []
        if has_mother:
            guardians.append({"person": mother_person.name, "relationship_with_student": "Mother"})
        if has_father:
            guardians.append({"person": father_person.name, "relationship_with_student": "Father"})
        sis_family = SISFamily({
            "home_address": row["address"],
            "children": [{"person": std_person.name}],
            "guardians": guardians
        })
        sis_family.save()


    # if len(families) > 1 and not family_found:
    #     for family in families:
    #         sis_family = SISFamily.find_by_id(family.name)
    #         print({
    #             "name": sis_family.name,
    #             "home_address": sis_family.home_address,
    #             "children": [SISPerson.find_by_id(child['person']).full_name for child in sis_family.children],
    #             "guardians": [SISPerson.find_by_id(guardian['person']).full_name for guardian in sis_family.guardians]
    #         })
    #         print("-------------------------------------------------")
    #     print(mother_person.to_json(), std_person.to_json())
    #     break


    # if family_id:
    #     sis_family = SISFamily.find_by_id(family_id)
    #     if (has_mother and ( not sis_family.guardian_exists(mother_person.name)) or (has_father and (not sis_family.guardian_exists(father_person.name)))):
    #         # Case 1: different family with the same address
    #         guardians = []
    #         if has_mother:
    #             guardians.append({"person": mother_person.name, "relationship_with_student": "Mother"})
    #         if has_father:
    #             guardians.append({"person": father_person.name, "relationship_with_student": "Father"})
            
    #         # create a hash for family address
    #         family_hash = abs(hash(row["address"])) % (10 ** 8)

    #         sis_family = SISFamily({
    #             "home_address": row["address"] + f" ({family_hash})",
    #             "children": [{"person": std_person.name}],
    #             "guardians": guardians
    #         })
    #         sis_family.save()
    #         print("--------------Family with same address", row["address"])
    #         break
    #     else:
    #         # Case 2: same family -> add student to children list, add mother and father to guardians list if not exists
    #         if has_mother and (not sis_family.guardian_exists(mother_person.name)):
    #             sis_family.guardians.append({"person": mother_person.name, "relationship_with_student": "Mother"})
    #         if has_father and (not sis_family.guardian_exists(father_person.name)):
    #             sis_family.guardians.append({"person": father_person.name, "relationship_with_student": "Father"})
    #         if not sis_family.child_exists(std_person.name):
    #             sis_family.children.append({"person": std_person.name})
    #         sis_family.save()
        
    # else:
        # guardians = []
        # if has_mother:
        #     guardians.append({"person": mother_person.name, "relationship_with_student": "Mother"})
        # if has_father:
        #     guardians.append({"person": father_person.name, "relationship_with_student": "Father"})
        # sis_family = SISFamily({
        #     "home_address": row["address"],
        #     "children": [{"person": std_person.name}],
        #     "guardians": guardians
        # })
        # sis_family.save()

In [ ]:
abs(hash('123')) % (10 ** 8)

### Course, Course Class, Course Class Person, Timetable

```
Subject Columns:  ['id', 'title', 'short_title']
School Class Columns:  ['id', 'title', 'short_title']
Course Class Columns:  ['grade_level', 'school_class', 'subject', 'course', 'course_class', 'teacher', 'email', 'wssg_id']
Timetable Columns:  ['Thứ', 'Thời gian', 'Tiết', 'Lớp 1.1\n (Ms.Dương)', 'Lớp 1.2\n (Ms. Trâm)', 'Lớp 1.3\n (Ms. Nguyệt)', 'Lớp 1.4\n (Ms. Duyên)', 'Lớp 1.5\n (Ms. Tuyền)', 'Lớp 1.6\n (Ms. Trang)', 'Lớp 1.7\n (Ms. Hà)']
```

TODO:
- [x] Get current school year id
- From timetable_id extract course_class and timetable_day_row_class:
    - [ ] Validation: subject must be in subject_df if the third column is a number
    - [ ] If the third column is not a number, it's a special course class (break, lunch, etc.)
    - [ ] Course Class Title = subject + grade_level + school_class
    - [ ] Get timetable_day_row_class (periods) from timetable_day_row_class
- For each row in course_class_df:
    - [ ] Create SIS Course, if not exists
    - [ ] Create SIS Course Class
    - [ ] Create SIS Course Class Person for teachers
    - [ ] Get student list from SIS School Class and add to SIS Course Class
    - [ ] Find timetable_day_row_class for each course_class
- [ ] Get student from all grade levels and add to special course classes

In [20]:
def create_course_classes(TIMETABLE_EXCEL_FILENAME, school_class_compare, timetable_type):
    sheet_names = pd.read_excel(TIMETABLE_EXCEL_FILENAME, sheet_name=None).keys()
    special_subjects = ["CLB", "Registration", "Snack", "Break", "Lunch", "TC1", "TC2", "TC3", "TC4"]

    not_found_subject = []
    found_subject = []
    for sheet_name in sheet_names:
        print("Processing sheet ", sheet_name)
        timetable_df = pd.read_excel(TIMETABLE_EXCEL_FILENAME, sheet_name=sheet_name, engine="openpyxl")
        timetable_df = normalize_df(timetable_df)

        for column in timetable_df.columns[3:]:
            school_class_short_title = extract_school_class_from_text(column)
            school_class = school_class_df[school_class_df["short_title"] == school_class_short_title]
            if school_class.empty:
                print("ERROR: Invalid school class", column)
            school_class = school_class.to_dict(orient="records")[0]['name']
            school_class = SISSchoolClass.find_by_id(school_class)
            print(f"Processing school class {school_class_short_title}")
    
            for index, row in timetable_df.iterrows():
                subject_title = row[column]
                subject_found = subject_df[subject_df[school_class_compare] == subject_title]
                if (subject_found.empty):
                    if (subject_title not in not_found_subject):
                        not_found_subject.append(subject_title)
                    
                    if subject_title in special_subjects:
                        course = SISCourse({
                            "title": f"{subject_title}",
                            "short_title": f"{subject_title}",
                            "description": "",
                            "program": "Other",
                        })
                        course_exists = course.save_if_not_exists(filters={"title": course.title})


                        course_class = SISCourseClass({
                            "school_year": CUR_SCHOOL_YEAR_ID,
                            "course": course.name,
                            "class_type": "Other",
                            "title": f'{subject_title}',
                            "short_title": f'{subject_title} {timetable_type}',
                        })
                        course_class_exists = course_class.save_if_not_exists(filters={"short_title": course_class.short_title})
                        course_class = SISCourseClass.find_by_id(course_class.name)


                        for person in school_class.participants:
                            person_exists = False
                            for participant in course_class.participants:
                                if participant['person'] == person['person']:
                                    person_exists = True
                                    break
                            if not person_exists:
                                course_class.participants.append({
                                    "person": person['person'],
                                    "role": "Student",
                                })
                        course_class.save()
                    else:
                        print(f"ERROR: Subject {subject_title} not found in special subjects! Class {school_class_short_title}, row {index}")
                        continue

                else:
                    subject_title = subject_found['title'].values[0]
                    if subject_title not in found_subject:
                        found_subject.append(subject_title)
                    
                    grade_level = school_class_short_title.split('.')[0]
                    course = SISCourse({
                        "title": f"{subject_title} K{grade_level}",
                        "short_title": f"{subject_found['short_title'].values[0]} K{grade_level}",
                        "description": "",
                        "program": subject_found['program'].values[0],
                    })
                    course_exists = course.save_if_not_exists(filters={"title": course.title})


                    course_class = SISCourseClass({
                        "from_school_class": school_class.name,
                        "school_year": CUR_SCHOOL_YEAR_ID,
                        "course": course.name,
                        "class_type": "School Class",
                        "enrollment_max": 25,
                        "enrollment_min": 0,
                        "participants": school_class.participants,
                        "title": f'{subject_title} K{grade_level} {school_class_short_title}',
                        "short_title": f'{subject_found["short_title"].values[0]} {school_class_short_title}',
                    })
                    course_class_exists = course_class.save_if_not_exists(filters={"title": course_class.title})
                    course_class = SISCourseClass.find_by_id(course_class.name)


                # Add timetable_day_row_class
                timetable_day = get_timetable_day(timetable.timetable_days, row.iloc[0])

                if not timetable_day:
                    raise ValueError(f"Invalid weekday: {row.iloc[0]}")
                
                start_time = row.iloc[1].split('-')[0].strip()
                end_time = row.iloc[1].split('-')[1].strip()
                for column_row in timetable_column.timetable_column_row:
                    # compare two string time e.g. '7:45' with '07:45:00'
                    if (datetime.strptime(column_row['time_start'], '%H:%M:%S').time() == datetime.strptime(start_time, '%H:%M').time()) and (datetime.strptime(column_row['time_end'], '%H:%M:%S').time() == datetime.strptime(end_time, '%H:%M').time()):
                        timetable_column_row = column_row
                        break


                period_exists = False
                for item in course_class.timetable_day_row_class:
                    if item['timetable_day'] == timetable_day['name'] and item['timetable_column_row'] == timetable_column_row['name']:
                        period_exists = True
                        break
                if not period_exists:
                    course_class.timetable_day_row_class.append({
                        "timetable_day": timetable_day['name'],
                        "timetable_column_row": timetable_column_row['name'],
                    })
                    course_class.save()

In [9]:
subject_df = pd.read_excel('../input_data/SIS Subject Updated.xlsx', engine="openpyxl")
subject_df = normalize_df(subject_df)

school_class_df = pd.read_excel('../input_data/SIS School Class.xlsx', engine="openpyxl")
school_class_df = normalize_df(school_class_df)

course_class_df = pd.read_excel('../input_data/SIS Course Class Updated.xlsx', engine="openpyxl")
course_class_df = normalize_df(course_class_df)

In [ ]:
# Extract data from timetable
# Step 1: Validation: subject must be in subject_df if the third column is a number

school_class_df = SISSchoolClass.find(limit_page_length=1000)
school_class_df['short_title'] = school_class_df['title'].apply(lambda x: x.split(' ')[1])

TIMETABLE_EXCEL_FILENAME = '../input_data/TKB_2324_Tieuhoc.xlsx'
sc_title = "title"


# TIMETABLE_EXCEL_FILENAME = '../input_data/TIMETABLE_2324_Trunghoc.xlsx'
# sc_title = "short_title"

# Go through all sheets in timetable excel file
sheet_names = pd.read_excel(TIMETABLE_EXCEL_FILENAME, sheet_name=None).keys()

not_found_subject = []
found_subject = []
for sheet_name in sheet_names:
    print("Processing sheet ", sheet_name)
    timetable_df = pd.read_excel(TIMETABLE_EXCEL_FILENAME, sheet_name=sheet_name, engine="openpyxl")
    timetable_df = normalize_df(timetable_df)
    
    for column in timetable_df.columns[3:]:
        school_class_short_title = extract_school_class_from_text(column)
        school_class_found = school_class_df[school_class_df["short_title"] == school_class_short_title].shape[0]
        if not school_class_found:
            print("ERROR: Invalid school class", column)
    
        for index, row in timetable_df.iterrows():
            # if pd.notna(row.iloc[2]) and (row.iloc[2] in ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']):
            subject_title = row[column]
            subject_found = subject_df[subject_df[sc_title] == subject_title].shape[0]
            if subject_found == 0 and subject_title not in not_found_subject:
                print(f"Subject |{subject_title}| not found in subject_df. Class {school_class_short_title}, row {index}")
                not_found_subject.append(subject_title)
            if subject_title not in found_subject:
                found_subject.append(subject_title)
            # TODO: Validate school class


#### Primary Timetable

In [ ]:
# Create Timetable TieuHoc

# Create timetable if not exists
timetable = SISTimetable({
    "grade_level_list": "1,2,3,4,5",
    "school_year": CUR_SCHOOL_YEAR_ID,
    "short_title": "TKB_TH",
    "status": "Active",
    "title": "Primary Timetable",
})
timetable.save_if_not_exists(filters={"title": timetable.title})

# Extract timetable column
TIMETABLE_EXCEL_FILENAME = '../input_data/TKB_2324_Tieuhoc.xlsx'
timetable_df = pd.read_excel(TIMETABLE_EXCEL_FILENAME, sheet_name=0, engine="openpyxl")
timetable_df = timetable_df[timetable_df.iloc[:,0]=='Ba']
timetable_column_dict = timetable_df.iloc[:, :3].values

type_dict = {
    "Registration": "Other",
    "Break": "Break",
    "Lunch": "Lunch",
    "Snack": "Snack",
    "Lesson": "Lesson"
}

timetable_column = SISTimetableColumn({
    "title": "TT Primary Weekday",
    "short_title": "TT Primary",
})
timetable_column.save_if_not_exists(filters={
    "title": timetable_column.title
})
timetable_column.timetable_column_row = [
    {
        "title": item[2] if item[2] not in list(range(11)) else f'Period {item[2]}',
        "short_title": item[2] if item[2] not in list(range(11)) else f'P{item[2]}',
        "type": type_dict[item[2]] if item[2] in type_dict.keys() else "Lesson" if item[2] in list(range(11)) else "Other",
        "time_start": item[1].split('-')[0].strip(),
        "time_end": item[1].split('-')[1].strip(),
    }
    for item in timetable_column_dict
]
timetable_column.save()

timetable.timetable_days = [
  { "title": "TKB_TH Monday", "short_title": "TKB_TH Mon", "timetable_column": timetable_column.name, "weekday": "Mon" },
  { "title": "TKB_TH Tuesday", "short_title": "TKB_TH Tue", "timetable_column": timetable_column.name, "weekday": "Tue" },
  { "title": "TKB_TH Wednesday", "short_title": "TKB_TH Wed", "timetable_column": timetable_column.name, "weekday": "Wed" },
  { "title": "TKB_TH Thursday", "short_title": "TKB_TH Thu", "timetable_column": timetable_column.name, "weekday": "Thu" },
  { "title": "TKB_TH Friday", "short_title": "TKB_TH Fri", "timetable_column": timetable_column.name, "weekday": "Fri" },
]
timetable.save()

timetable = SISTimetable.find_by_id(timetable.name)
timetable_column = SISTimetableColumn.find_by_id(timetable_column.name)

In [ ]:
delete_all(SISCourseClass)

In [ ]:
# Create Course Class for Primary

sheet_names = pd.read_excel(TIMETABLE_EXCEL_FILENAME, sheet_name=None).keys()
special_subjects = ["CLB", "Registration", "Snack", "Break", "Lunch", "TC1", "TC2", "TC3", "TC4"]

not_found_subject = []
found_subject = []
for sheet_name in sheet_names:
    print("Processing sheet ", sheet_name)
    timetable_df = pd.read_excel(TIMETABLE_EXCEL_FILENAME, sheet_name=sheet_name, engine="openpyxl")
    timetable_df = normalize_df(timetable_df)

    for column in timetable_df.columns[3:]:
        school_class_short_title = extract_school_class_from_text(column)
        school_class = school_class_df[school_class_df["short_title"] == school_class_short_title]
        if school_class.empty:
            print("ERROR: Invalid school class", column)
        school_class = school_class.to_dict(orient="records")[0]['name']
        school_class = SISSchoolClass.find_by_id(school_class)
        print("Processing school class ", school_class_short_title)
    
        for index, row in timetable_df.iterrows():
            # if pd.notna(row.iloc[2]) and (row.iloc[2] in ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']):
            subject_title = row[column]
            subject_found = subject_df[subject_df["title"] == subject_title]
            if (subject_found.empty):
                if (subject_title not in not_found_subject):
                    not_found_subject.append(subject_title)
                    print(f"Subject |{subject_title}| not found in subject_df. Class {school_class_short_title}, row {index}")
                
                if subject_title in special_subjects:
                    course = SISCourse({
                        "title": f"{subject_title}",
                        "short_title": f"{subject_title}",
                        "description": "",
                        "program_type": "Other",
                    })
                    course.save_if_not_exists(filters={"title": course.title})

                    course_class = SISCourseClass({
                        "school_year": CUR_SCHOOL_YEAR_ID,
                        "course": course.name,
                        "class_type": "Other",
                        "title": f'{subject_title}',
                        "short_title": f'{subject_title} Primary',
                    })
                    course_class.save_if_not_exists(filters={"short_title": course_class.short_title})
                    course_class = SISCourseClass.find_by_id(course_class.name)

                    for person in school_class.participants:
                        person_exists = False
                        for participant in course_class.participants:
                            if participant['person'] == person['person']:
                                person_exists = True
                                break
                        if not person_exists:
                            course_class.participants.append({
                                "person": person['person'],
                                "role": "Student",
                            })
                    course_class.save()
                else:
                    print(f"ERROR: Subject {subject_title} not found! Class {school_class_short_title}, row {index}")
                    continue

            else:
                if subject_title not in found_subject:
                    found_subject.append(subject_title)
                
                grade_level = school_class_short_title.split('.')[0]
                course = SISCourse({
                    "title": f"{subject_title} K{grade_level}",
                    "short_title": f"{subject_found['short_title'].values[0]} K{grade_level}",
                    "description": "",
                    "program_type": subject_found['program'].values[0],
                })
                course.save_if_not_exists(filters={"title": course.title})

                course_class = SISCourseClass({
                    "from_school_class": school_class.name,
                    "school_year": CUR_SCHOOL_YEAR_ID,
                    "course": course.name,
                    "class_type": "School Class",
                    "enrollment_max": 25,
                    "enrollment_min": 0,
                    "participants": school_class.participants,
                    "title": f'{subject_title} K{grade_level} {school_class_short_title}',
                    "short_title": f'{subject_found["short_title"].values[0]} K{grade_level} {school_class_short_title}',
                })
                course_class.save_if_not_exists(filters={"title": course_class.title})
                course_class = SISCourseClass.find_by_id(course_class.name)

            # Add timetable_day_row_class
            timetable_day = get_timetable_day(timetable.timetable_days, row.iloc[0])
            if not timetable_day:
                raise ValueError(f"Invalid weekday: {row.iloc[0]}")
            
            start_time = row.iloc[1].split('-')[0].strip()
            end_time = row.iloc[1].split('-')[1].strip()
            for column_row in timetable_column.timetable_column_row:
                if column_row['time_start'].startswith(start_time) and column_row['time_end'].startswith(end_time):
                    timetable_column_row = column_row
                    break

            period_exists = False
            for item in course_class.timetable_day_row_class:
                if item['timetable_day'] == timetable_day['name'] and item['timetable_column_row'] == timetable_column_row['name']:
                    period_exists = True
                    break
            if not period_exists:
                course_class.timetable_day_row_class.append({
                    "timetable_day": timetable_day['name'],
                    "timetable_column_row": timetable_column_row['name'],
                })
                course_class.save()

In [ ]:
# Validate data
course_class_list = SISCourseClass.find(limit_page_length=2000, output="Object")

participant_count = {}
for course_class in course_class_list:
    cc = SISCourseClass.find_by_id(course_class.name)
    pc = len(cc.participants)
    participant_count[cc.title] = pc

In [ ]:
# Count total number of primary student
count = 0
for school_class_id in school_class_df['name'].values:
    school_class = SISSchoolClass.find_by_id(school_class_id)
    if school_class.short_title.split('.')[0] in ['1', '2', '3', '4', '5']:
        print(f"School class {school_class.title}: {len(school_class.participants)} students")
        count += len(school_class.participants)
print(f"Total number of primary student: {count}")

#### Secondary Timetable

In [12]:
# Create Timetable TieuHoc

# Create timetable if not exists
timetable = SISTimetable({
    "grade_level_list": "6,7,8,9,10,11,12",
    "school_year": CUR_SCHOOL_YEAR_ID,
    "short_title": "TKB_TrH",
    "status": "Active",
    "title": "Secondary Timetable",
})
timetable.save_if_not_exists(filters={"title": timetable.title})

# Extract timetable column
TIMETABLE_EXCEL_FILENAME = '../input_data/TIMETABLE_2324_Trunghoc.xlsx'
timetable_df = pd.read_excel(TIMETABLE_EXCEL_FILENAME, sheet_name=0, engine="openpyxl")
timetable_df = timetable_df[timetable_df.iloc[:,0]=='Monday']
timetable_column_dict = timetable_df.iloc[:, :3].values

type_dict = {
    "Registration": "Other",
    "Break": "Break",
    "Lunch": "Lunch",
    "Snack": "Snack",
    "Lesson": "Lesson"
}

timetable_column = SISTimetableColumn({
    "title": "TT Secondary Weekday",
    "short_title": "TT Secondary",
})
timetable_column.save_if_not_exists(filters={
    "title": timetable_column.title
})

timetable_column = SISTimetableColumn.find_by_id(timetable_column.name)
if len(timetable_column.timetable_column_row) == 0:
    timetable_column.timetable_column_row = [
        {
            "title": item[2] if item[2] not in list(range(11)) else f'Period {item[2]}',
            "short_title": item[2] if item[2] not in list(range(11)) else f'P{item[2]}',
            "type": type_dict[item[2]] if item[2] in type_dict.keys() else "Lesson" if item[2] in list(range(11)) else "Other",
            "time_start": item[1].split('-')[0].strip(),
            "time_end": item[1].split('-')[1].strip(),
        }
        for item in timetable_column_dict
    ]
    timetable_column.save()

timetable = SISTimetable.find_by_id(timetable.name)
if len(timetable.timetable_days) == 0:
    timetable.timetable_days = [
    { "title": "TKB_TrH Monday", "short_title": "TKB_TrH Mon", "timetable_column": timetable_column.name, "weekday": "Mon" },
    { "title": "TKB_TrH Tuesday", "short_title": "TKB_TrH Tue", "timetable_column": timetable_column.name, "weekday": "Tue" },
    { "title": "TKB_TrH Wednesday", "short_title": "TKB_TrH Wed", "timetable_column": timetable_column.name, "weekday": "Wed" },
    { "title": "TKB_TrH Thursday", "short_title": "TKB_TrH Thu", "timetable_column": timetable_column.name, "weekday": "Thu" },
    { "title": "TKB_TrH Friday", "short_title": "TKB_TrH Fri", "timetable_column": timetable_column.name, "weekday": "Fri" },
    ]
    timetable.save()

timetable = SISTimetable.find_by_id(timetable.name)
timetable_column = SISTimetableColumn.find_by_id(timetable_column.name)

In [16]:
# Create Course Class for Secondary
TIMETABLE_EXCEL_FILENAME = '../input_data/TIMETABLE_2324_Trunghoc.xlsx'
school_class_compare = 'short_title'
timetable_type = 'Secondary'

timetable = SISTimetable.find_one(filters={'title': 'Secondary Timetable'})
timetable_column = SISTimetableColumn.find_one(filters={'title': 'TT Secondary Weekday'})

school_class_df = SISSchoolClass.find(limit_page_length=1000)
school_class_df['short_title'] = school_class_df['title'].apply(lambda x: x.split(' ')[1])

create_course_classes(TIMETABLE_EXCEL_FILENAME, school_class_compare, timetable_type)

Processing sheet  Sheet1
Processing school class 6.1
Processing school class 6.2
Processing school class 6.3
Processing school class 6.4
Processing school class 6.5
Processing school class 6.6
Processing school class 6.7
Processing school class 6.8
Processing school class 6.9
Processing school class 6.10
Processing school class 7.1
Processing school class 7.2
Processing school class 7.3
Processing school class 7.4
Processing school class 7.5
Processing school class 7.6
Processing school class 7.7
Processing school class 8.1
Processing school class 8.2
Processing school class 8.3
Processing school class 8.4
Processing school class 8.5
Processing school class 9.1
Processing school class 9.2
Processing school class 9.3
Processing school class 9.4
Processing school class 9.5
Processing school class 10.1
Processing school class 10.2
Processing school class 10.3
Processing school class 10.4
Processing school class 11.1
Processing school class 11.2
Processing school class 11.3
Processing scho


SISTimetable:
name: aoijgpccka; grade_level_list: 1,2,3,4,5; school_year: 15af1fa3b3; short_title: TKB_TH; status: Active; timetable_days: [{'name': 'h1frucm8a2', 'owner': 'Administrator', 'creation': '2024-07-25 17:05:34.754424', 'modified': '2024-07-25 22:48:28.304585', 'modified_by': 'Administrator', 'docstatus': 0, 'idx': 1, 'title': 'TKB_TH Monday', 'short_title': 'TKB_TH Mon', 'timetable_column': 'bef6ij8apf', 'weekday': 'Mon', 'parent': 'aoijgpccka', 'parentfield': 'timetable_days', 'parenttype': 'SIS Timetable', 'doctype': 'SIS Timetable Day'}, {'name': 'h1fr8tq6p5', 'owner': 'Administrator', 'creation': '2024-07-25 17:05:34.754424', 'modified': '2024-07-25 22:48:28.304585', 'modified_by': 'Administrator', 'docstatus': 0, 'idx': 2, 'title': 'TKB_TH Tuesday', 'short_title': 'TKB_TH Tue', 'timetable_column': 'bef6ij8apf', 'weekday': 'Tue', 'parent': 'aoijgpccka', 'parentfield': 'timetable_days', 'parenttype': 'SIS Timetable', 'doctype': 'SIS Timetable Day'}, {'name': 'h1fr6m3kli

In [24]:
# Create Course Class for Secondary
TIMETABLE_EXCEL_FILENAME = '../input_data/TKB_2324_Tieuhoc.xlsx'
school_class_compare = 'title'
timetable_type = 'Primary'

timetable = SISTimetable.find_one(filters={'title': 'Primary Timetable'})
timetable = SISTimetable.find_by_id(timetable.name)
timetable_column = SISTimetableColumn.find_one(filters={'title': 'TT Primary Weekday'})
timetable_column = SISTimetableColumn.find_by_id(timetable_column.name)

school_class_df = SISSchoolClass.find(limit_page_length=1000)
school_class_df['short_title'] = school_class_df['title'].apply(lambda x: x.split(' ')[1])

create_course_classes(TIMETABLE_EXCEL_FILENAME, school_class_compare, timetable_type)

Processing sheet  Sheet1
Processing school class 1.1
Processing school class 1.2
Processing school class 1.3
Processing school class 1.4
Processing school class 1.5
Processing school class 1.6
Processing school class 1.7
Processing sheet  Sheet2
Processing school class 2.1
Processing school class 2.2
Processing school class 2.3
Processing school class 2.4
Processing school class 2.5
Processing school class 2.6
Processing school class 2.7
Processing sheet  Sheet3
Processing school class 3.1
Processing school class 3.2
Processing school class 3.3
Processing school class 3.4
Processing school class 3.5
Processing school class 3.6
Processing school class 3.7
Processing sheet  Sheet4
Processing school class 4.1
Processing school class 4.2
Processing school class 4.3
Processing school class 4.4
Processing school class 4.5
Processing school class 4.6
Processing school class 4.7
Processing sheet  Sheet5
Processing school class 5.1
Processing school class 5.2
Processing school class 5.3
Process

In [ ]:
# course_df = SISCourse.find_all()

# for index, row in course_df.iterrows():
#     course = SISCourse.find_by_id(row['name'])
#     program = subject_df[subject_df['title'].apply(lambda subject: course.title.startswith(subject))]['program'].values
#     program = program[0] if len(program) > 0 else "Other"
#     course.program_type = program
#     course.save()
    
# course_df.to_excel('../input_data/sis_course.xlsx', index=False)

In [26]:
course_classes = SISCourseClass.find_all()

for index, row in course_classes.iterrows():
    course_class = SISCourseClass.find_by_id(row['name'])
    for subject in ['Âm Nhạc', 'Mỹ Thuật']:
        for grade_level in [6,7,8]:
            if course_class.title.startswith(f'{subject} K{grade_level}'):
                course_class.title = course_class.title.replace(f'{subject}', f'Nghệ Thuật')
                course_class.short_title = course_class.short_title.replace(f"{subject if subject=='Mỹ Thuật' else 'Nhạc'}", f'Nghệ Thuật')
                # print(course_class.title, course_class.short_title)
                course_class.save()

### Import teachers

In [ ]:
teacher_df = pd.read_csv('../input_data/SIS Person (Teacher) Final.csv', sep=';')
teacher_df = teacher_df[teacher_df['employee_code'].notna()]
teacher_df['date_of_birth'] = teacher_df['date_of_birth'].apply(lambda x: datetime.strftime(datetime.strptime(x, "%d/%m/%Y"), "%Y-%m-%d"))
teacher_df['gender'] = teacher_df['gender'].apply(lambda x: "Male" if x == "M" else "Female")

In [ ]:
for index, row in teacher_df.iterrows():
    print("Executing row ", index)
    # Create SIS Person for student, if not exists
    first_name, last_name = split_name(row['full_name'])
    tea_person = SISPerson({
        "first_name": first_name,
        "last_name": last_name,
        "email": row['email'],
        "phone_number": row['mobile_phone_number'],
        "gender": row["gender"],
        # "date_of_birth": datetime.strftime(row["std_dob"], "%Y-%m-%d"),
        "date_of_birth": row['date_of_birth'],
        "primary_role": "Teacher",
    })
    tea_person_exists = tea_person.save_if_not_exists(filters={
        "email": row['email'],
    })

    # Create SIS Student with person id
    sis_student = SISStaff({
        "person": tea_person.name,
        "employee_code": row["employee_code"],
        "department": row["department"],
    })
    sis_student.save_if_not_exists(filters={
        "employee_code": row["employee_code"]
    })

In [37]:

school_classes = SISSchoolClass.find_all()
course_classes = SISCourseClass.find(limit_page_length=2000)
teachers = SISStaff.find_all()

In [57]:
course_classes.shape

(1194, 19)

In [26]:
# Validate data
# Check if all value of course_class_df['school_class'] is in school_classes['title']
for index, row in course_class_df.iterrows():
    if row['school_class'] not in school_classes['title'].values:
        print(f"ERROR: Invalid school class {row['school_class']}")

In [56]:
course_classes[course_classes['short_title'].str.contains("5.1")]

,name,owner,creation,modified,modified_by,docstatus,idx,school_year,course,title,short_title,attendance,enrollment_min,enrollment_max,class_type,get_from,from_school_class,from_grade_level,timetable
1041,4nfu9mmj7p,Administrator,2024-07-27 21:51:18.198553,2024-07-27 21:52:25.820208,Administrator,0,0,15af1fa3b3,h6366jg0om,ELA K5 5.1,ELA K5 5.1,0,0,25,School Class,School Class,None,None,None
1042,4ngtu6psh5,Administrator,2024-07-27 21:51:21.387802,2024-07-27 21:52:34.821197,Administrator,0,0,15af1fa3b3,h63aam6ba3,Toán K5 5.1,Toán K5 5.1,0,0,25,School Class,School Class,None,None,None
1043,4nh0qt6bfj,Administrator,2024-07-27 21:51:21.633039,2024-07-27 21:52:35.024265,Administrator,0,0,15af1fa3b3,h63d5g8hmm,Tiếng Việt K5 5.1,Tiếng Việt K5 5.1,0,0,25,School Class,School Class,None,None,None
1044,4ni1m8hpp5,Administrator,2024-07-27 21:51:24.922296,2024-07-27 21:52:26.053709,Administrator,0,0,15af1fa3b3,h63hr57cnc,Khoa Học K5 5.1,Khoa Học K5 5.1,0,0,25,School Class,School Class,None,None,None
1045,4ni3gsijso,Administrator,2024-07-27 21:51:25.162425,2024-07-27 21:51:25.282991,Administrator,0,0,15af1fa3b3,h63k4c909i,AC K5 5.1,AC K5 5.1,0,0,25,School Class,School Class,None,None,None
1046,4nj1jtjcs9,Administrator,2024-07-27 21:51:28.133582,2024-07-27 21:52:07.222606,Administrator,0,0,15af1fa3b3,h63nk09hln,Math K5 5.1,Math K5 5.1,0,0,25,School Class,School Class,None,None,None
1047,4nj6jh1ldq,Administrator,2024-07-27 21:51:28.614765,2024-07-27 21:51:28.746368,Administrator,0,0,15af1fa3b3,h63rjcfbor,SHCN K5 5.1,SHCN K5 5.1,0,0,25,School Class,School Class,None,None,None
1048,4nl3ujjfh9,Administrator,2024-07-27 21:51:34.761555,2024-07-27 21:51:34.895314,Administrator,0,0,15af1fa3b3,h642gnd029,Lịch Sử K5 5.1,Sử K5 5.1,0,0,25,School Class,School Class,None,None,None
1049,4nl6j73g42,Administrator,2024-07-27 21:51:35.031430,2024-07-27 21:51:35.387741,Administrator,0,0,15af1fa3b3,h6451uge10,GDTC K5 5.1,GDTC K5 5.1,0,0,25,School Class,School Class,None,None,None
1050,4nm5gah6h9,Administrator,2024-07-27 21:51:38.155216,2024-07-27 21:51:38.291752,Administrator,0,0,15af1fa3b3,h6491c9o90,GDCX K5 5.1,GDCX K5 5.1,0,0,25,School Class,School Class,None,None,None


In [73]:
for index, row in course_classes.iterrows():
    subject_title = row['title'].split(' ')[:-2]
    subject_title = ' '.join(subject_title)
    class_title = row['title'].split(' ')[-2:]
    class_title = ' '.join(class_title)

    subject_found = subject_df[subject_df['short_title'] == subject_title]
    if (not subject_found.empty) and (subject_title != subject_found['title'].values[0]):
        print('Changing title', row['title'], 'to', f"{subject_found['title'].values[0]} {class_title}", end='...')
        course_class = SISCourseClass.find_by_id(row['name'])
        course_class.title = f"{subject_found['title'].values[0]} {class_title}"
        course_class.save()
        print('Done')
        

Changing title Văn K6 6.1 to Ngữ Văn K6 6.1...Done
Changing title Anh K6 6.1 to Ngoại Ngữ 1 K6 6.1...Done
Changing title LS & ĐL K6 6.1 to Lịch Sử Và Địa Lý K6 6.1...Done
Changing title KHTN K6 6.1 to Khoa Học Tự Nhiên K6 6.1...Done
Changing title CN K6 6.1 to Công Nghệ K6 6.1...Done
Changing title Nhạc K6 6.1 to Âm Nhạc K6 6.1...Done
Changing title KHTN K6 6.2 to Khoa Học Tự Nhiên K6 6.2...Done
Changing title Nhạc K6 6.2 to Âm Nhạc K6 6.2...Done
Changing title Văn K6 6.2 to Ngữ Văn K6 6.2...Done
Changing title CN K6 6.2 to Công Nghệ K6 6.2...Done
Changing title LS & ĐL K6 6.2 to Lịch Sử Và Địa Lý K6 6.2...Done
Changing title Anh K6 6.2 to Ngoại Ngữ 1 K6 6.2...Done
Changing title KHTN K6 6.3 to Khoa Học Tự Nhiên K6 6.3...Done
Changing title Anh K6 6.3 to Ngoại Ngữ 1 K6 6.3...Done
Changing title Nhạc K6 6.3 to Âm Nhạc K6 6.3...Done
Changing title CN K6 6.3 to Công Nghệ K6 6.3...Done
Changing title LS & ĐL K6 6.3 to Lịch Sử Và Địa Lý K6 6.3...Done
Changing title Văn K6 6.3 to Ngữ Văn K6 

In [36]:
# for index, row in course_classes[course_classes['title'].str.contains("Thể Dục K11")].iterrows():
#     course_class = SISCourseClass.find_by_id(row['name'])
#     course_class.title = course_class.title.replace("Thể Dục", "GDQP/ GDTC")
#     course_class.save()

# course_classes[course_classes['title'].str.contains("Thể Dục K10")]

In [40]:
course_class_df = pd.read_csv('../input_data/SIS Course Class Updated.csv', sep=';', encoding='utf-8')
# course_class_df = pd.read_excel('../input_data/SIS Course Class Updated.xlsx', engine="openpyxl")
course_class_df = normalize_df(course_class_df)
course_class_df = course_class_df[course_class_df['email'].notna()]

# Check if all value of course_class_df['course_class'] is in course_classes['title']
# count = 0
# for index, row in course_class_df.iterrows():
#     if row['course_class'] not in course_classes['title'].values:
#         print(f"ERROR: Invalid course class {row['course_class']} at row {index}")
#         count += 1
# print(f"Total number of invalid course class: {count}")

# Add teacher to course_class if found in course_class_df
for index, row in course_classes.iterrows():
    if row['title'] not in course_class_df['course_class'].values:
        # print(f"ERROR: Invalid course class {title}")
        continue
    try:
        teacher_email = course_class_df[course_class_df['course_class'] == row['title']]['email'].values[0]
        teacher_person = SISPerson.find_one(filters={"email": teacher_email})
    except Exception as e:
        print(f"ERROR: Teacher not found {teacher_email}")
        continue
    
    course_class = SISCourseClass.find_by_id(row['name'])

    person_exists = False
    for participant in course_class.participants:
        if participant['person'] == person['person']:
            person_exists = True
            break
    if not person_exists:
        course_class.participants.append({
            "person": teacher_person.name,
            "role": "Teacher",
        })
        course_class.save()


ERROR: Teacher not found daniel.leblanc@wellspringsaigon.edu.vn
ERROR: Teacher not found daniel.leblanc@wellspringsaigon.edu.vn
ERROR: Teacher not found daniel.leblanc@wellspringsaigon.edu.vn


In [ ]:




# Check if all value of course_class_df['email'] is in teachers['email']
for index, row in course_class_df.iterrows():
    if row['email'] not in teachers['email'].values:
        print(f"ERROR: Invalid teacher email {row['email']}")
        break
    teacher = teachers[teachers['email'] == row['email']]
    if row['wssg_id'] != teacher['employee_code'].values[0]:
        print(f"ERROR: Invalid teacher employee code {row['wssg_id']}")
    if row['teacher'] != teacher['full_name'].values[0]:
        print(f"ERROR: Invalid teacher name {row['teacher']}")

### Create User Account for guardians and teacher

In [53]:
teachers = SISPerson.find(filters={"primary_role": "Teacher"}, limit_page_length=2000)
guardians = SISPerson.find(filters={"primary_role": "Guardian"}, limit_page_length=2000)

In [100]:
guardians[guardians['email'].str.contains("tth0408198")]['email'].values

array([' tth04081989@gmail.com'], dtype=object)

In [84]:
employee_parent = [
    "Lưu Mỹ Hạnh",
    "Hồ Thị Bích Vân",
    "Phạm Phương Lan",
    "Nguyễn Thị Thanh Trâm",
    "Trần Thị Khánh Linh",
    "Phạm Ngọc Ánh",
    "Đỗ Thu Thủy",
    "Phạm Thị Như Ý",
    "Nguyễn Thị Ngọc Huân",
    "Phạm Thị Bảo Ngọc",
    "Nguyễn Thị Minh Nguyệt",
    "Vũ Ngọc Đan Thanh",
    "Ngô Thị Hồng Anh",
    "Ninh Thị Nga",
    "Nguyễn Lê Ngọc Anh",
    "Văn Thị Thanh Sự",
    "Nguyễn Thị Phương Hạnh",
    "Nguyễn Thị Diệu Hiền",
    "Vũ Ngọc Đan Thanh",
    "Cory James Engle",
    "Trần Nguyễn Ngọc Thảo",
    "Clerkin James Jonathan",
]
print(len(employee_parent))

guardians_full_name = guardians[guardians['full_name'].isin(employee_parent)]['full_name'].values

# find names in employee_parent but not in guardians
for name in employee_parent:
    if name not in guardians_full_name:
        print(name)

guardians = guardians[guardians['full_name'].isin(employee_parent)]
print("Limit to ", len(guardians), "guardians")

22
Ninh Thị Nga
Nguyễn Thị Diệu Hiền
Cory James Engle
Limit to  18 guardians


In [86]:
guardians[['full_name', 'phone_number', 'email']].to_csv('../input_data/employee_parent.csv', index=False)
# guardians[['full_name', 'phone_number', 'email']]

In [80]:
SISFamilyGuardian.find_by_id('8iig1nrome')


SISFamilyGuardian:
name: ; person: ; relationship_with_student: ; notes: ; child_data_access: ; parent: ; 

In [81]:
guardians[guardians['full_name'].str.contains("Ninh")]

,name,owner,creation,modified,modified_by,docstatus,idx,avatar,first_name,middle_name,last_name,full_name,english_name,common_name,email,phone_number,date_of_birth,gender,nationality,primary_role
19,8dod9dl3u0,Administrator,2024-07-25 14:57:53.340086,2024-07-25 14:57:53.340086,Administrator,0,9,None,Hoàng Ninh,,Lê,Lê Hoàng Ninh,None,None,None,0963604604,None,Male,None,Guardian
536,8gkg4un5sl,Administrator,2024-07-25 15:02:48.031083,2024-07-25 15:02:48.031083,Administrator,0,5,None,Ninh,,Vũ,Vũ Ninh,None,None,vuninh@gmail.com,0903834785,None,Male,None,Guardian
1248,8hohsq4b6e,Administrator,2024-07-25 15:04:43.299496,2024-07-25 15:04:43.299496,Administrator,0,5,None,Thị Ninh,,Bùi,Bùi Thị Ninh,None,None,bui.thi.ninh@gmail.com,0938150708,None,Female,None,Guardian


In [72]:
guardians.head()

,name,owner,creation,modified,modified_by,docstatus,idx,avatar,first_name,middle_name,last_name,full_name,english_name,common_name,email,phone_number,date_of_birth,gender,nationality,primary_role
0,8cr8jrejlf,Administrator,2024-07-25 14:56:20.012043,2024-07-25 14:56:20.012043,Administrator,0,8,None,Thị Thu,,Đỗ,Đỗ Thị Thu,None,None,dokyva@gmail.com,0384803568,None,Female,None,Guardian
1,8cr8k9sq8p,Administrator,2024-07-25 14:56:20.042622,2024-07-25 14:56:20.042622,Administrator,0,8,None,Thanh Phúc,,Trần,Trần Thanh Phúc,None,None,phuc149@gmail.com,0909748149,None,Male,None,Guardian
2,8cvoo2fc5s,Administrator,2024-07-25 14:56:34.473646,2024-07-25 14:56:34.473646,Administrator,0,10,None,Phương Trúc Vy,,Trần,Trần Phương Trúc Vy,None,None,trucvy0708@yahoo.com,0906155188,None,Female,None,Guardian
3,8cvp1i8tql,Administrator,2024-07-25 14:56:34.528447,2024-07-25 14:56:34.528447,Administrator,0,10,None,Trọng Đạt,,Nguyễn,Nguyễn Trọng Đạt,None,None,trucvy8578@gmail.com,0909020981,None,Male,None,Guardian
4,8dnrd3dji3,Administrator,2024-07-25 14:57:51.550391,2024-07-25 14:57:51.550391,Administrator,0,6,None,Lê Thanh,,Trần,Trần Lê Thanh,None,None,thanhbinhdat@gmail.com,0933280876,None,Female,None,Guardian


In [97]:
from user import User
from pp_user import PPUser
import time

def create_account_for_person(person, sis_role):
    if (not person["email"]):
        print("Email is required")
        return
    if (person["primary_role"] == 'Student'):
        print("Student cannot have account")
        return
    
    pp_user = PPUser.find_by_id(person["email"])
    if pp_user and getattr(pp_user, "name", None):
        print(f"PP User {pp_user.name} already exists")
        return
    # account = User.find_by_id(person["email"])
    # if account and ("PP User" in account.roles) and ("System Manager" in account.roles):
    #     return
    
    # print(f'Creating account for {person["full_name"]} - {person["email"]} - {person["primary_role"]}')
    account = User({
        "email": person["email"],
        "first_name": person["first_name"],
        "last_name": person["last_name"],
        "send_welcome_email": 0,
    })
    account.save_if_not_exists(filters={"email": account.email})

    account = User.find_by_id(account.name)
    account.roles = [{"role": "System Manager"}, {"role": "PP User"}]
    account.new_password = "wellspring@2024"
    account.save()

    # account.reset_password()
    # account.update_password('wellspring@2024')

    # Create PP User
    pp_user = PPUser({
        "enabled": 1,
        "person": person["name"],
        "user": account.name,
        "sis_role": sis_role,  # ["Parent", "Teacher", "Admin"]
    })
    pp_user.save_if_not_exists(filters={"name": account.name})

# total_step = teachers.shape[0]
# step = 0
# for index, row in teachers.iterrows():
#     create_account_for_person(row, "Teacher")
#     step += 1
#     if step % 25 == 0:
#         print(f"Processed {step}/{total_step}...", end='')
#         time.sleep(5)
#         print("Done")
    

# for index, row in guardians.iterrows():
#     create_account_for_person(row, "Parent")

In [105]:
total_step = guardians.shape[0]
step = 0
for index, row in guardians.iterrows():
    create_account_for_person(row, "Parent")
    step += 1
    if step % 25 == 0:
        print(f"Processed {step}/{total_step}...", end='')
        time.sleep(5)
        print("Done")

In [51]:
from user import User

test_acc = User({
    "email": "minh.do@wellspringsaigon.edu.vn",
    "first_name": "Minh",
    "last_name": "Do",
    "send_welcome_email": 0,
    "set_new_password": "wellspring@2024",
})
test_acc.save_if_not_exists(filters={"email": test_acc.email})
test_acc.roles = [{"role": "System Manager"}, {"role": "PP User"}]
test_acc.new_password = 'abc@abc'
test_acc.save()

# test_acc = User.find_by_id('minh.do@wellspringsaigon.edu.vn')
# test_acc.reset_password()
# test_acc.update_password('wellspring@2024')

In [49]:
test_acc = User.find_by_id('dhminh1024@gmail.com')
test_acc.reset_password()


'http://localhost:8000/parent_portal/reset-password/7871d64b61c72c5ec0ca86b106d402739c22b97d7752d9b3784d6779'

In [ ]:
test_acc.reset_password_key

In [ ]:
test_acc.update_password('wellspring@2024')

In [ ]:
users = User.find_all()
mia = users[users['email'] == 'mia.do@wellspringsaigon.edu.vn'].to_dict()
mia.keys()

In [ ]:
users.columns

In [ ]:
test_acc.reset_password_key